# **Stock market prediction**
# using PyTorch RNN and LSTM

In [60]:
!pip install torch

## Import data

In [61]:
# importing necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [62]:
data = pd.read_csv('GOOG.csv')
data

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,GOOG,2016-06-14 00:00:00+00:00,718.27,722.470,713.1200,716.48,1306065,718.27,722.470,713.1200,716.48,1306065,0.0,1.0
1,GOOG,2016-06-15 00:00:00+00:00,718.92,722.980,717.3100,719.00,1214517,718.92,722.980,717.3100,719.00,1214517,0.0,1.0
2,GOOG,2016-06-16 00:00:00+00:00,710.36,716.650,703.2600,714.91,1982471,710.36,716.650,703.2600,714.91,1982471,0.0,1.0
3,GOOG,2016-06-17 00:00:00+00:00,691.72,708.820,688.4515,708.65,3402357,691.72,708.820,688.4515,708.65,3402357,0.0,1.0
4,GOOG,2016-06-20 00:00:00+00:00,693.71,702.480,693.4100,698.77,2082538,693.71,702.480,693.4100,698.77,2082538,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,GOOG,2021-06-07 00:00:00+00:00,2466.09,2468.000,2441.0725,2451.32,1192453,2466.09,2468.000,2441.0725,2451.32,1192453,0.0,1.0
1254,GOOG,2021-06-08 00:00:00+00:00,2482.85,2494.495,2468.2400,2479.90,1253253,2482.85,2494.495,2468.2400,2479.90,1253253,0.0,1.0
1255,GOOG,2021-06-09 00:00:00+00:00,2491.40,2505.000,2487.3300,2499.50,1006337,2491.40,2505.000,2487.3300,2499.50,1006337,0.0,1.0
1256,GOOG,2021-06-10 00:00:00+00:00,2521.60,2523.260,2494.0000,2494.01,1561733,2521.60,2523.260,2494.0000,2494.01,1561733,0.0,1.0


In [63]:
data.columns

Index(['symbol', 'date', 'close', 'high', 'low', 'open', 'volume', 'adjClose',
       'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'],
      dtype='object')

In [64]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=data['date'],
                open= data['open'],
                high= data['high'],
                low= data['low'],
                close= data['close'])])
fig.show()

In [65]:
data = data[['date','close']]
data

,date,close
0,2016-06-14 00:00:00+00:00,718.27
1,2016-06-15 00:00:00+00:00,718.92
2,2016-06-16 00:00:00+00:00,710.36
3,2016-06-17 00:00:00+00:00,691.72
4,2016-06-20 00:00:00+00:00,693.71
...,...,...
1253,2021-06-07 00:00:00+00:00,2466.09
1254,2021-06-08 00:00:00+00:00,2482.85
1255,2021-06-09 00:00:00+00:00,2491.40
1256,2021-06-10 00:00:00+00:00,2521.60


In [66]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

C:\Users\Rashi Bhatnagar\AppData\Local\Temp\ipykernel_20992\2670177637.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [67]:
data['date'] = pd.to_datetime(data['date'])

plt.plot(data['date'], data['close'])

## Data Preprocessing

In [68]:
from copy import deepcopy as dc

def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    df.set_index('date', inplace=True)

    for i in range(1, n_steps+1):
        df[f'close(t-{i})'] = df['close'].shift(i)

    df.dropna(inplace=True)

    return df

lookback = 7
shifted_df = prepare_dataframe_for_lstm(data, lookback)
shifted_df

,close,close(t-1),close(t-2),close(t-3),close(t-4),close(t-5),close(t-6),close(t-7)
date,,,,,,,,
2016-06-23 00:00:00+00:00,701.87,697.46,695.94,693.71,691.72,710.36,718.92,718.27
2016-06-24 00:00:00+00:00,675.22,701.87,697.46,695.94,693.71,691.72,710.36,718.92
2016-06-27 00:00:00+00:00,668.26,675.22,701.87,697.46,695.94,693.71,691.72,710.36
2016-06-28 00:00:00+00:00,680.04,668.26,675.22,701.87,697.46,695.94,693.71,691.72
2016-06-29 00:00:00+00:00,684.11,680.04,668.26,675.22,701.87,697.46,695.94,693.71
...,...,...,...,...,...,...,...,...
2021-06-07 00:00:00+00:00,2466.09,2451.76,2404.61,2421.28,2429.81,2411.56,2402.51,2433.53
2021-06-08 00:00:00+00:00,2482.85,2466.09,2451.76,2404.61,2421.28,2429.81,2411.56,2402.51
2021-06-09 00:00:00+00:00,2491.40,2482.85,2466.09,2451.76,2404.61,2421.28,2429.81,2411.56


In [69]:
shifted_df_as_np = shifted_df.to_numpy()

shifted_df_as_np

array([[ 701.87,  697.46,  695.94, ...,  710.36,  718.92,  718.27],
       [ 675.22,  701.87,  697.46, ...,  691.72,  710.36,  718.92],
       [ 668.26,  675.22,  701.87, ...,  693.71,  691.72,  710.36],
       ...,
       [2491.4 , 2482.85, 2466.09, ..., 2421.28, 2429.81, 2411.56],
       [2521.6 , 2491.4 , 2482.85, ..., 2404.61, 2421.28, 2429.81],
       [2513.93, 2521.6 , 2491.4 , ..., 2451.76, 2404.61, 2421.28]])

In [70]:
shifted_df_as_np.shape

(1251, 8)

In [71]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
shifted_df_as_np = scaler.fit_transform(shifted_df_as_np)

shifted_df_as_np

array([[-0.96373035, -0.96848932, -0.96963481, ..., -0.95278946,
        -0.94260368, -0.94334011],
       [-0.99248924, -0.96373035, -0.96796735, ..., -0.97369218,
        -0.95230191, -0.94260368],
       [-1.        , -0.99248924, -0.96312955, ..., -0.97146061,
        -0.9734205 , -0.95230191],
       ...,
       [ 0.96741019,  0.9581836 ,  0.97223472, ...,  0.96582002,
         0.99578535,  0.97510862],
       [ 1.        ,  0.96741019,  0.99062058, ...,  0.94712644,
         0.9861211 ,  0.99578535],
       [ 0.99172305,  1.        ,  1.        , ...,  1.        ,
         0.96723447,  0.9861211 ]])

In [72]:
X = shifted_df_as_np[:, 1:]
y = shifted_df_as_np[:, 0]

X.shape, y.shape

((1251, 7), (1251,))

In [73]:
X = dc(np.flip(X, axis=1))
X

array([[-0.94334011, -0.94260368, -0.95278946, ..., -0.97194959,
        -0.96963481, -0.96848932],
       [-0.94260368, -0.95230191, -0.97369218, ..., -0.96949173,
        -0.96796735, -0.96373035],
       [-0.95230191, -0.9734205 , -0.97146061, ..., -0.96781642,
        -0.96312955, -0.99248924],
       ...,
       [ 0.97510862,  0.99578535,  0.96582002, ...,  0.96573331,
         0.97223472,  0.9581836 ],
       [ 0.99578535,  0.9861211 ,  0.94712644, ...,  0.98152751,
         0.99062058,  0.96741019],
       [ 0.9861211 ,  0.96723447,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

In [74]:
split_index = int(len(X) * 0.95)

split_index

1188

### Train test split

In [75]:
X_train = X[:split_index]
X_test = X[split_index:]

y_train = y[:split_index]
y_test = y[split_index:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1188, 7), (63, 7), (1188,), (63,))

In [76]:
X_train = X_train.reshape((-1, lookback, 1))
X_test = X_test.reshape((-1, lookback, 1))

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1188, 7, 1), (63, 7, 1), (1188, 1), (63, 1))

In [77]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([1188, 7, 1]),
 torch.Size([63, 7, 1]),
 torch.Size([1188, 1]),
 torch.Size([63, 1]))

### Defining train and test loaders

In [86]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        x = torch.tensor(self.X[i], dtype=torch.float32)  # Assuming self.data[index] is your input data
        y = torch.tensor(self.y[i], dtype=torch.long) 
        return x,y

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [87]:
train_dataset

In [88]:
from torch.utils.data import DataLoader
from torch.utils.data._utils.collate import default_collate

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=default_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,collate_fn=default_collate)

In [89]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 7, 1]) torch.Size([16, 1])


C:\Users\Rashi Bhatnagar\AppData\Local\Temp\ipykernel_20992\2703000592.py:12: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Rashi Bhatnagar\AppData\Local\Temp\ipykernel_20992\2703000592.py:13: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



## Implementing RNN

In [94]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, states = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = RNN(1, 4, 1).to(device)
model

RNN(
  (rnn): RNN(1, 4, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [95]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [96]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)
    if batch_index % 10 == 1:
      print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
      print('***************************************************')
      print()

In [97]:
learning_rate = 0.001
num_epochs = 100
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1


C:\Users\Rashi Bhatnagar\AppData\Local\Temp\ipykernel_20992\2703000592.py:12: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Rashi Bhatnagar\AppData\Local\Temp\ipykernel_20992\2703000592.py:13: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



AttributeError: 'tuple' object has no attribute 'dim'

In [ ]:
with torch.no_grad():
    predicted = model(X_train.to(device)).to('cpu').numpy()

plt.plot(y_train, label='Actual Close')
plt.plot(predicted, label='Predicted Close')
plt.xlabel('Day')
plt.ylabel('Close')
plt.legend()
plt.show()

In [ ]:
train_predictions = predicted.flatten()

dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = train_predictions
dummies = scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])

In [ ]:
dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = y_test.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test

In [ ]:
plt.plot(new_y_test, label='Actual Close')
plt.plot(test_predictions, label='Predicted Close')
plt.xlabel('Day')
plt.ylabel('Close')
plt.legend()
plt.show()

## Implementing LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 4, 1)
model.to(device)
model

In [ ]:
learning_rate = 0.001
num_epochs = 100
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

In [ ]:
with torch.no_grad():
    predicted = model(X_train.to(device)).to('cpu').numpy()

plt.plot(y_train, label='Actual Close')
plt.plot(predicted, label='Predicted Close')
plt.xlabel('Day')
plt.ylabel('Close')
plt.legend()
plt.show()


In [ ]:
train_predictions = predicted.flatten()

dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = train_predictions
dummies = scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])
train_predictions

In [ ]:
dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = y_train.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_train = dc(dummies[:, 0])
new_y_train

In [ ]:
plt.plot(new_y_train, label='Actual Close')
plt.plot(train_predictions, label='Predicted Close')
plt.xlabel('Day')
plt.ylabel('Close')
plt.legend()
plt.show()


In [ ]:
test_predictions = model(X_test.to(device)).detach().cpu().numpy().flatten()

dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = test_predictions
dummies = scaler.inverse_transform(dummies)

test_predictions = dc(dummies[:, 0])
test_predictions

In [ ]:
dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = y_test.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test

In [ ]:
plt.plot(new_y_test, label='Actual Close')
plt.plot(test_predictions, label='Predicted Close')
plt.xlabel('Day')
plt.ylabel('Close')
plt.legend()
plt.show()
